In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats, optimize
import seaborn as sns

In [2]:
n_rounds = 1000
time = np.linspace(0,24,n_rounds)

np.random.uniform(0,24)

dates = pd.date_range('2020-01-01', '2020-01-02', freq='S')

In [3]:
morning_size = dates[dates.hour<=6].size
probs = np.concatenate([np.repeat(0.1, morning_size)/morning_size, np.repeat(0.9, dates.size-morning_size)/(dates.size-morning_size)])
times = np.sort(np.random.choice(dates, p=probs, size=n_rounds))
df = pd.DataFrame(times)

# Reserve prices for revenue increase

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats, optimize
import seaborn as sns

In [5]:
N_rounds = 1000
N_objects = 100
N_bidders = 130

budgets = np.array([  30.,   50.,   80.,  100.,  120.,  200.,  300.,  500., 1000.])
budgets_p = np.ones(9)/9
budgets_init = np.array([np.random.choice(budgets, p=budgets_p) for _ in range(N_bidders)])

fees = np.array([0, 0.3, 0.36, 0.68])
fees_p = np.array([0.36, 0.14, 0.30, 0.2])
fees_init = np.array([np.random.choice(fees, p=fees_p) for i in range(N_objects)])

auctioned_objects = np.array([AuctionedObject(i, 2.5, np.inf, fee=fees_init[i]) for i in range(N_objects)])
auctioneer = Auctioneer(auctioned_objects, x0 = np.ones(N_objects)*0)
bidders = np.array([SimpleBidder(budget=budgets_init[i]) for i in range(N_bidders)])

NameError: name 'AuctionedObject' is not defined

In [ ]:
def myerson_rp(house):
    x0 = house.auctioneer.x0[house.auctioned_object.id_]
    dist = stats.lognorm(s=0.4, loc=-7, scale=16)
    fn = lambda r: r-(1-dist.cdf(r))/dist.pdf(r)-x0
    r_star = optimize.fsolve(fn, x0=0.3)
    return r_star

def rp_fees(house):
    obj_id = house.auctioned_object.id_
    x0 = house.auctioneer.x0[obj_id]
    fee = house.auctioned_object.fee
    r = fee*np.mean([house.winning_bid , house.second_bid]) + (1-fee)*x0
    return r 

house = AuctionHouse(N_rounds, 
                     auctioneer, 
                     bidders, 
                     rp_policy=myerson_rp
                    )

house.run()

In [ ]:
df = house.bidder_tracker.make_dataFrame()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(14,4))
df['bids'].mean(axis=1).plot(ax=ax1)
df['budgets'].mean(axis=1).plot(ax=ax2)

In [ ]:
auc_df = house.auction_tracker.make_dataFrame()
auc_df

# Throttling

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

from AuctionSimulator.Agents.Bidders import SimpleBidder
from AuctionSimulator.Agents.AuctionedObject import AuctionedObject, Auctioneer
from AuctionSimulator.Controllers import AuctionHouse

from AuctionSimulator.Policies.Throttling import Throttling

## test single policy

In [ ]:
N_rounds = 1000
N_objects = 100
N_bidders = 130

budgets = np.array([  30.,   50.,   80.,  100.,  120.,  200.,  300.,  500., 1000.])
budgets_p = np.ones(9)/9
budgets_init = np.array([np.random.choice(budgets, p=budgets_p) for _ in range(N_bidders)])

fees = np.array([0, 0.3, 0.36, 0.68])
fees_p = np.array([0.36, 0.14, 0.30, 0.2])
fees_init = np.array([np.random.choice(fees, p=fees_p) for i in range(N_objects)])

throttling = Throttling(N_rounds, budgets_init, planning='uniform', probability='linear',
                        probability_kwargs={'floor': 0.})

In [ ]:
auctioned_objects = np.array([AuctionedObject(i, 2, np.inf, fee=fees_init[i]) for i in range(N_objects)])
auctioneer = Auctioneer(auctioned_objects)
bidders = np.array([SimpleBidder(budget=budgets_init[i]) for i in range(N_bidders)])

house = AuctionHouse(N_rounds, 
                     auctioneer, 
                     bidders, 
                     rp_policy=None,
                     throttling=throttling
                    )

house.run()

In [ ]:
df = house.bidder_tracker.make_dataframe()

In [ ]:
df['bids'].mean(axis=1).rolling(10).mean().plot()

## test multiple policies

In [ ]:
def test_policy(throttling, fees_init, budgets_init):
    auctioned_objects = np.array([AuctionedObject(i, 1, np.inf, fee=fees_init[i]) for i in range(N_objects)])
    auctioneer = Auctioneer(auctioned_objects)
    bidders = np.array([SimpleBidder(budget=budgets_init[i]) for i in range(N_bidders)])

    house = AuctionHouse(N_rounds, 
                         auctioneer, 
                         bidders, 
                         rp_policy=None,
                         throttling=throttling,
                        )

    for i in range(N_rounds):
        obj_id = house.select_object_to_sell()
        house.sell_object(obj_id)
        
    return house

In [ ]:
N_rounds = 1000
N_objects = 100
N_bidders = 130

budgets = np.array([30, 50, 80])
budgets_p = np.array([0.4, 0.4, 0.2])
budgets_init = np.array([np.random.choice(budgets, p=budgets_p) for _ in range(N_bidders)])

fees = np.array([0, 0.3, 0.7])
fees_p = np.array([0.5, 0.25, 0.25])
fees_init = np.array([np.random.choice(fees, p=fees_p) for i in range(N_objects)])

uniform_throttling = Throttling(N_rounds, budgets_init)
sigmoid_throttling1 = Throttling(N_rounds, budgets_init, planning='sigmoid', planning_kwargs={'s':1, 't0':0.5})
sigmoid_throttling2 = Throttling(N_rounds, budgets_init, planning='sigmoid', planning_kwargs={'s':1, 't0':1})

throttling_policies = [uniform_throttling, sigmoid_throttling1, sigmoid_throttling2]
N_policies = len(throttling_policies)

In [ ]:
%%time
N_days = 200
revenues_data = np.zeros((N_days, N_policies))
fees_paid_data = np.zeros((N_days, N_policies))

for thr, throttling in enumerate(throttling_policies):
    print('testing policy ', thr)
    for d in range(N_days):
        house = test_policy(throttling, fees_init, budgets_init)
        revenues_data[d, thr] = house.auctioneer.revenue
        fees_paid_data[d, thr] = house.auctioneer.fees_paid

In [ ]:
throttling.total_probability??

In [ ]:
revenues_data.mean(axis=0).round(2)

In [ ]:
fees_paid_data.mean(axis=0).round(2)

# Fees reduction by dynamic reserve price

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

from Auction_simulator.agents import AuctionedObject, Auctioneer, SimpleBidder, AuctionHouse
from Auction_simulator.trackers import AuctionTracker, BidderTracker
from Auction_simulator.auction_types import SecondPriceAuction

In [ ]:
def rp_fees1(house):
    if house.auctioned_object.fee > 0:
        return np.mean([house.second_bid, house.winning_bid])
    else:
        return house.auctioneer.x0
    
def rp_fees2(house):
    fee = house.auctioned_object.fee
    r = fee*house.winning_bid + (1-fee)*house.second_bid
    return r 
    
def rp_fees3(house):
    fee = house.auctioned_object.fee
    r = fee*np.mean([house.winning_bid + house.second_bid]) + (1-fee)*house.auctioneer.x0
    return r 

In [ ]:
def test_policy(rp_policy, fees_init, budgets_init):

    auctioned_objects = np.array([AuctionedObject(i, 1, np.inf, fee=fees_init[i]) for i in range(N_objects)])
    auctioneer = Auctioneer(auctioned_objects, x0=1)
    bidders = np.array([SimpleBidder(budget=budgets_init[i]) for i in range(N_bidders)])

    auction_tracker = AuctionTracker(N_rounds)
    bidder_tracker = BidderTracker(N_rounds, N_bidders, N_objects)
    house = AuctionHouse(auctioneer, bidders, 
                         rp_policy = rp_fees1,
                         auction_tracker=auction_tracker,
                         bidder_tracker=bidder_tracker
                        )

    for i in range(N_rounds):
        obj_id = house.select_object_to_sell()
        house.sell_object(obj_id)
        
    return house

In [ ]:
N_rounds = 1000
N_objects = 100
N_bidders = 130

budgets = np.array([30, 50, 80])
budgets_p = np.array([0.4, 0.4, 0.2])
budgets_init = np.array([np.random.choice(budgets, p=budgets_p) for _ in range(N_bidders)])

fees = np.array([0, 0.3, 0.7])
fees_p = np.array([0.5, 0.25, 0.25])
fees_init = np.array([np.random.choice(fees, p=fees_p) for i in range(N_objects)])

In [ ]:
%%time

rp_policies = [None, rp_fees1, rp_fees2, rp_fees3]
rp_policies = [None]
N_policies = len(rp_policies)
N_days = 1
revenues_data = np.zeros((N_days, N_policies))
fees_paid_data = np.zeros((N_days, N_policies))

for rp, rp_policy in enumerate(rp_policies):
    print('testing policy ', rp)
    for d in range(N_days):
        house = test_policy(rp_policy, fees_init, budgets_init)
        revenues_data[d, rp] = house.auctioneer.revenue
        fees_paid_data[d, rp] = house.auctioneer.fees_paid

In [ ]:
revenues_data.mean(axis=0).round(2)

In [ ]:
fees_paid_data.mean(axis=0).round(2)